In [ ]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
from llama_index.llms.bedrock import Bedrock

llm = Bedrock(model="anthropic.claude-3-sonnet-20240229-v1:0")     

#response = llm.complete("Paul Graham is ")

In [ ]:
import json
from llama_index.embeddings.bedrock import BedrockEmbedding

embed_model = BedrockEmbedding()

supported_models = BedrockEmbedding.list_supported_models()

print(json.dumps(supported_models, indent=2))

In [ ]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader("./data").load_data()

In [ ]:
filenames = ["Sach-EA-2023-04-06-Police-Version Nr 23(FLExA).pdf"]

documents = []
for filename in filenames:
    doc = SimpleDirectoryReader(input_files=[f"data/{filename}"]).load_data()
    doc[0].doc_id = filename.replace(".pdf", "")
    documents.extend(doc)

In [ ]:
documents

In [ ]:
from llama_index.core import PropertyGraphIndex

index = PropertyGraphIndex.from_documents(
    documents,
    llm=llm,
    embed_model=embed_model,
    show_progress=True,
)

In [ ]:
index.property_graph_store.save_networkx_graph(name="./knowledge_graph.html")

In [ ]:
from llama_index.core.indices.property_graph import (
    ImplicitPathExtractor,
    SimpleLLMPathExtractor,
)

index = PropertyGraphIndex.from_documents(
    documents,
    embed_model=embed_model,
    kg_extractors=[
        ImplicitPathExtractor(),
        SimpleLLMPathExtractor(
            llm=llm,
            num_workers=4,
            max_paths_per_chunk=10,
        ),
    ],
    show_progress=True,
)

In [ ]:
retriever = index.as_retriever(
        include_text=False,  # include source text, default True
        similarity_top_k=2
)

#prompt = "What is the purpose of Agile?"
#prompt = "Wenn Agilität nicht das primäre Ziel verfolgt effektiver zu sein, eine höhere Ausnutzung zu erzielen und Kosten zu senken, was bringt es mir dann?"
#prompt = "Wie ist der Name des Versicherungsnehmers?"
#prompt = "Was sind die Versicherten?"
#prompt = "Ist Feuer als Gefahr versichert?"
#prompt = "Ist Sturm als Gefahr versichert oder nicht versichert?"
#prompt = "Sind Kosten auf erstes Risiko versichert?"
prompt = "Wie hoch ist die Selbstbeteiligung (SB) bei Feuer als Gefahr?"

nodes = retriever.retrieve(prompt)

for node in nodes:
    print(node.text)

In [ ]:
# response = query_engine.query(prompt)

# print(str(response))

In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
#from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from IPython.display import Markdown, display
import chromadb

In [ ]:
# create client and a new collection
chroma_client = chromadb.EphemeralClient()
chroma_collection = chroma_client.create_collection("quickstart")

# define embedding function
# embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")

# load documents
documents = SimpleDirectoryReader("./data").load_data()

# set up ChromaVectorStore and load in data
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, embed_model=embed_model
)

In [ ]:
#prompt = "What is the purpose of Agile?"
#prompt = "Wenn Agilität nicht das primäre Ziel verfolgt effektiver zu sein, eine höhere Ausnutzung zu erzielen und Kosten zu senken, was bringt es mir dann?"
#prompt = "Wie ist der Name des Versicherungsnehmers?"
#prompt = "Was sind die Versicherten?"
#prompt = "Ist Feuer als Gefahr versichert?"
#prompt = "Ist Sturm als Gefahr versichert oder nicht versichert?"
#prompt = "Sind Kosten auf erstes Risiko versichert?"
prompt = "Wie hoch ist die Selbstbeteiligung (SB) bei Feuer als Gefahr?"

In [ ]:
results = index.as_retriever().retrieve(prompt)

for i, node in enumerate(results):
    print(i)
    display(Markdown(f"{node.text}"))

https://github.com/aws-samples/amazon-bedrock-rag-workshop/blob/main/05_Semantic_Search_with_Reranking/01_LlamaIndex_Reranker_Bedrock_Claude.ipynb

In [ ]:
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    ServiceContext,
    #LLMPredictor,
    get_response_synthesizer,
    set_global_service_context,
    StorageContext,
    ListIndex
)

In [ ]:
import sys

from llama_index.indices.postprocessor import LLMRerank
from IPython.display import Markdown, display

In [ ]:
from llama_index.llms.bedrock import Bedrock
from llama_index.embeddings.bedrock import BedrockEmbedding
from llama_index.core import ServiceContext, set_global_service_context

llm = Bedrock(model="anthropic.claude-3-sonnet-20240229-v1:0")

embed_model = BedrockEmbedding().from_credentials(
    model_name='amazon.titan-embed-g1-text-02'

)
chunk_overlap = 20
chunk_size = 512
service_context = ServiceContext.from_defaults(llm=llm, 
                                               embed_model=embed_model, 
                                               chunk_size=chunk_size,
                                               chunk_overlap=chunk_overlap,
                                            )

set_global_service_context(service_context)

In [ ]:
index = VectorStoreIndex.from_documents(documents, 
                                        service_context=service_context)

In [ ]:
from llama_index import Document
from llama_index.node_parser import SentenceSplitter

node_parser = SentenceSplitter(chunk_size=512, chunk_overlap=20)

nodes = node_parser.get_nodes_from_documents(
    documents
    , show_progress=True
)

In [ ]:
# initialize storage context (by default it's in-memory)
storage_context = StorageContext.from_defaults()
storage_context.docstore.add_documents(nodes)

In [ ]:
from llama_index.prompts.base import Prompt
from llama_index.prompts.prompt_type import PromptType

CLAUDE_CHOICE_SELECT_PROMPT_TMPL = (
    """
    
    Human: A list of documents is shown below. Each document has a number next to it along  with a summary of the document. A question is also provided. 
    Respond with the respective document number. You should consult to answer the question, in order of relevance, as well as the relevance score. 
    The relevance score is a number from 1-10 based on how relevant you think the document is to the question.\n"
    Do not include any documents that are not relevant to the question. 
    
    Example format: 
    Document 1:\n<summary of document 1>
    Document 2:\n<summary of document 2>
    ...\n\n
    Document 10:\n<summary of document 10>
    
    Question: <question>
    Answer:
    Doc: 9, Relevance: 7
    Doc: 3, Relevance: 4
    Doc: 7, Relevance: 3

    Let's try this now: 
    {context_str}

    Question: {query_str}
    
    Assistant: Answer:"""
)

claude_choice_select_prompt = Prompt(
    CLAUDE_CHOICE_SELECT_PROMPT_TMPL, prompt_type=PromptType.CHOICE_SELECT
)

In [ ]:
from llama_index.retrievers import VectorIndexRetriever
from llama_index.indices.query.schema import QueryBundle
import pandas as pd
from IPython.display import display, HTML

pd.set_option("display.max_colwidth", None)

def get_retrieved_nodes(
    query_str, vector_top_k=10, reranker_top_n=3, with_reranker=False
):
    query_bundle = QueryBundle(query_str)
    # configure retriever
    retriever = VectorIndexRetriever(
        index=index,
        similarity_top_k=vector_top_k,
        choice_select_prompt=claude_choice_select_prompt

    )
    
    retrieved_nodes = retriever.retrieve(query_bundle)

    if with_reranker:
        # configure reranker
        reranker = LLMRerank(
            choice_batch_size=5, 
            top_n=reranker_top_n, 
            service_context=service_context, 
            choice_select_prompt=claude_choice_select_prompt

        )
        retrieved_nodes = reranker.postprocess_nodes(retrieved_nodes, query_bundle)

    return retrieved_nodes